In [37]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn import model_selection as grid_search
import xgboost as xgb

In [2]:
train_df = pd.read_csv('data/train_users_2.csv',header=0, index_col=None)
test_df = pd.read_csv('data/test_users.csv', header=0, index_col=None)

In [3]:
#combine
all_df = pd.concat((train_df, test_df), axis=0, ignore_index=True)

In [6]:
train_df.set_index('id', inplace=True)
train_df.head()

,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
id,,,,,,,,,,,,,,,
gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


In [8]:
test_df.set_index('id', inplace=True)
test_df.head()

,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser
id,,,,,,,,,,,,,,
5uwns89zht,2014-07-01,20140701000006,NaN,FEMALE,35.0,facebook,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
jtl0dijy2j,2014-07-01,20140701000051,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
xx0ulgorjt,2014-07-01,20140701000148,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome
6c6puo6ix0,2014-07-01,20140701000215,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,IE
czqhjk3yfe,2014-07-01,20140701000305,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Safari


In [9]:
all_df.set_index('id', inplace=True)
all_df.head()

,affiliate_channel,affiliate_provider,age,country_destination,date_account_created,date_first_booking,first_affiliate_tracked,first_browser,first_device_type,gender,language,signup_app,signup_flow,signup_method,timestamp_first_active
id,,,,,,,,,,,,,,,
gxn3p5htnn,direct,direct,NaN,NDF,2010-06-28,NaN,untracked,Chrome,Mac Desktop,-unknown-,en,Web,0,facebook,20090319043255
820tgsjxq7,seo,google,38.0,NDF,2011-05-25,NaN,untracked,Chrome,Mac Desktop,MALE,en,Web,0,facebook,20090523174809
4ft3gnwmtx,direct,direct,56.0,US,2010-09-28,2010-08-02,untracked,IE,Windows Desktop,FEMALE,en,Web,3,basic,20090609231247
bjjt8pjhuk,direct,direct,42.0,other,2011-12-05,2012-09-08,untracked,Firefox,Mac Desktop,FEMALE,en,Web,0,facebook,20091031060129
87mebub9p4,direct,direct,41.0,US,2010-09-14,2010-02-18,untracked,Chrome,Mac Desktop,-unknown-,en,Web,0,basic,20091208061105


# Clean Data

- Clean timestamp
- Remove booking date field
- Clean age column
- identify and fill missing data

In [10]:
# Clean timestamp
all_df['date_account_created'] = pd.to_datetime(all_df['date_account_created'], format='%Y-%m-%d')
all_df['timestamp_first_active'] = pd.to_datetime(all_df['timestamp_first_active'], format='%Y%m%d%H%M%S')
all_df['date_account_created'].fillna(all_df.timestamp_first_active, inplace=True)

In [11]:
# timestamp_first_active,date_account_created, and date_account_created' fixed
all_df.head()

,affiliate_channel,affiliate_provider,age,country_destination,date_account_created,date_first_booking,first_affiliate_tracked,first_browser,first_device_type,gender,language,signup_app,signup_flow,signup_method,timestamp_first_active
id,,,,,,,,,,,,,,,
gxn3p5htnn,direct,direct,NaN,NDF,2010-06-28,NaN,untracked,Chrome,Mac Desktop,-unknown-,en,Web,0,facebook,2009-03-19 04:32:55
820tgsjxq7,seo,google,38.0,NDF,2011-05-25,NaN,untracked,Chrome,Mac Desktop,MALE,en,Web,0,facebook,2009-05-23 17:48:09
4ft3gnwmtx,direct,direct,56.0,US,2010-09-28,2010-08-02,untracked,IE,Windows Desktop,FEMALE,en,Web,3,basic,2009-06-09 23:12:47
bjjt8pjhuk,direct,direct,42.0,other,2011-12-05,2012-09-08,untracked,Firefox,Mac Desktop,FEMALE,en,Web,0,facebook,2009-10-31 06:01:29
87mebub9p4,direct,direct,41.0,US,2010-09-14,2010-02-18,untracked,Chrome,Mac Desktop,-unknown-,en,Web,0,basic,2009-12-08 06:11:05


In [12]:
# Remove date_first_booking column
all_df.drop('date_first_booking', axis=1, inplace=True)

In [13]:
all_df.head()

,affiliate_channel,affiliate_provider,age,country_destination,date_account_created,first_affiliate_tracked,first_browser,first_device_type,gender,language,signup_app,signup_flow,signup_method,timestamp_first_active
id,,,,,,,,,,,,,,
gxn3p5htnn,direct,direct,NaN,NDF,2010-06-28,untracked,Chrome,Mac Desktop,-unknown-,en,Web,0,facebook,2009-03-19 04:32:55
820tgsjxq7,seo,google,38.0,NDF,2011-05-25,untracked,Chrome,Mac Desktop,MALE,en,Web,0,facebook,2009-05-23 17:48:09
4ft3gnwmtx,direct,direct,56.0,US,2010-09-28,untracked,IE,Windows Desktop,FEMALE,en,Web,3,basic,2009-06-09 23:12:47
bjjt8pjhuk,direct,direct,42.0,other,2011-12-05,untracked,Firefox,Mac Desktop,FEMALE,en,Web,0,facebook,2009-10-31 06:01:29
87mebub9p4,direct,direct,41.0,US,2010-09-14,untracked,Chrome,Mac Desktop,-unknown-,en,Web,0,basic,2009-12-08 06:11:05


In [14]:
# Remove outliers function
def remove_outliers(df, column, min_val, max_val):
    col_values = df[column].values
    df[column] = np.where(np.logical_or(col_values<=min_val, col_values>=max_val), np.NaN, col_values)
    return df

# clean age column
all_df = remove_outliers(df=all_df, column='age', min_val=15, max_val=90)
all_df['age'].fillna(-1, inplace=True)

/home/salas/miniconda3/envs/kaggle/lib/python3.5/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in less_equal
  after removing the cwd from sys.path.
/home/salas/miniconda3/envs/kaggle/lib/python3.5/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in greater_equal
  after removing the cwd from sys.path.


In [15]:
all_df.head()

,affiliate_channel,affiliate_provider,age,country_destination,date_account_created,first_affiliate_tracked,first_browser,first_device_type,gender,language,signup_app,signup_flow,signup_method,timestamp_first_active
id,,,,,,,,,,,,,,
gxn3p5htnn,direct,direct,-1.0,NDF,2010-06-28,untracked,Chrome,Mac Desktop,-unknown-,en,Web,0,facebook,2009-03-19 04:32:55
820tgsjxq7,seo,google,38.0,NDF,2011-05-25,untracked,Chrome,Mac Desktop,MALE,en,Web,0,facebook,2009-05-23 17:48:09
4ft3gnwmtx,direct,direct,56.0,US,2010-09-28,untracked,IE,Windows Desktop,FEMALE,en,Web,3,basic,2009-06-09 23:12:47
bjjt8pjhuk,direct,direct,42.0,other,2011-12-05,untracked,Firefox,Mac Desktop,FEMALE,en,Web,0,facebook,2009-10-31 06:01:29
87mebub9p4,direct,direct,41.0,US,2010-09-14,untracked,Chrome,Mac Desktop,-unknown-,en,Web,0,basic,2009-12-08 06:11:05


In [16]:
# fill missing data
all_df['first_affiliate_tracked'].fillna(-1, inplace=True)

In [17]:
all_df.head()

,affiliate_channel,affiliate_provider,age,country_destination,date_account_created,first_affiliate_tracked,first_browser,first_device_type,gender,language,signup_app,signup_flow,signup_method,timestamp_first_active
id,,,,,,,,,,,,,,
gxn3p5htnn,direct,direct,-1.0,NDF,2010-06-28,untracked,Chrome,Mac Desktop,-unknown-,en,Web,0,facebook,2009-03-19 04:32:55
820tgsjxq7,seo,google,38.0,NDF,2011-05-25,untracked,Chrome,Mac Desktop,MALE,en,Web,0,facebook,2009-05-23 17:48:09
4ft3gnwmtx,direct,direct,56.0,US,2010-09-28,untracked,IE,Windows Desktop,FEMALE,en,Web,3,basic,2009-06-09 23:12:47
bjjt8pjhuk,direct,direct,42.0,other,2011-12-05,untracked,Firefox,Mac Desktop,FEMALE,en,Web,0,facebook,2009-10-31 06:01:29
87mebub9p4,direct,direct,41.0,US,2010-09-14,untracked,Chrome,Mac Desktop,-unknown-,en,Web,0,basic,2009-12-08 06:11:05


# Transformation & Feature Extraction
- enhance the data for classification algorithms
- 

## Data transformation
- bucket/binning (numerical data)
    - instead of age we use "buckets" 15-35 years
- normalization
    - convert values with given std, from average.
- logarithmic functions
- [one hot encoding](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f)(categorical)
    - convert categories to columns with binary values (0 false, 1 for true) of that given column.
    - easier for algorithms

## Feature extraction
- Using Higherarchical Information
- adding external data

---

### Transform categorical data
**One Hot Encoding** – replacing the categorical fields in the dataset with multiple columns representing one value from each column.


In [18]:
def convert_to_binary(df, column_to_convert):
    '''function that you input a dataframe and the colum to convert,
    and creates binary columns of the categories.'''
    categories = list(df[column_to_convert].drop_duplicates())
    
    for category in categories:
        cat_name = str(category).replace(" ", "_").replace("(", "").replace(")", "").replace("/", "_").replace("-", "").lower()
        col_name = column_to_convert[:5] + '_' + cat_name[:10]
        df[col_name] = 0
        df.loc[(df[column_to_convert] == category), col_name] = 1
    return df

In [19]:
# One Hot Encoding
columns_to_convert = ['gender', 'signup_method', 'signup_flow', 'language', 'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser']

for column in columns_to_convert:
    all_df = convert_to_binary(df=all_df, column_to_convert=column)
    all_df.drop(column, axis=1, inplace=True)

In [20]:
all_df.head()

,age,country_destination,date_account_created,timestamp_first_active,gende_unknown,gende_male,gende_female,gende_other,signu_facebook,signu_basic,...,first_theworld_b,first_slimbrowse,first_epic,first_stainless,first_googlebot,first_outlook_20,first_icedragon,first_ibrowse,first_nintendo_b,first_uc_browser
id,,,,,,,,,,,,,,,,,,,,,
gxn3p5htnn,-1.0,NDF,2010-06-28,2009-03-19 04:32:55,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
820tgsjxq7,38.0,NDF,2011-05-25,2009-05-23 17:48:09,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4ft3gnwmtx,56.0,US,2010-09-28,2009-06-09 23:12:47,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
bjjt8pjhuk,42.0,other,2011-12-05,2009-10-31 06:01:29,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
87mebub9p4,41.0,US,2010-09-14,2009-12-08 06:11:05,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


## Create New Features
- extract info from `date_account_created` and `timestamp_first_active` into its own columns, then delete originals

In [21]:
# Add new date related fields

# date_account_created extraction
all_df['day_account_created'] = all_df['date_account_created'].dt.weekday
all_df['month_account_created'] = all_df['date_account_created'].dt.month
all_df['quarter_account_created'] = all_df['date_account_created'].dt.quarter
all_df['year_account_created'] = all_df['date_account_created'].dt.year

# timestamp_first_active extraction
all_df['hour_first_active'] = all_df['timestamp_first_active'].dt.hour
all_df['day_first_active'] = all_df['timestamp_first_active'].dt.weekday
all_df['month_first_active'] = all_df['timestamp_first_active'].dt.month
all_df['quarter_first_active'] = all_df['timestamp_first_active'].dt.quarter
all_df['year_first_active'] = all_df['timestamp_first_active'].dt.year

all_df['created_less_active'] = (all_df['date_account_created'] - all_df['timestamp_first_active']).dt.days

# Drop unnecessary columns
columns_to_drop = ['date_account_created', 'timestamp_first_active', 'date_first_booking', 'country_destination']
for column in columns_to_drop:
    if column in all_df.columns:
        all_df.drop(column, axis=1, inplace=True)

In [23]:
# from 14 col to 163 col!
all_df.head()

,age,gende_unknown,gende_male,gende_female,gende_other,signu_facebook,signu_basic,signu_google,signu_weibo,signu_0,...,day_account_created,month_account_created,quarter_account_created,year_account_created,hour_first_active,day_first_active,month_first_active,quarter_first_active,year_first_active,created_less_active
id,,,,,,,,,,,,,,,,,,,,,
gxn3p5htnn,-1.0,1,0,0,0,1,0,0,0,1,...,0,6,2,2010,4,3,3,1,2009,465
820tgsjxq7,38.0,0,1,0,0,1,0,0,0,1,...,2,5,2,2011,17,5,5,2,2009,731
4ft3gnwmtx,56.0,0,0,1,0,0,1,0,0,0,...,1,9,3,2010,23,1,6,2,2009,475
bjjt8pjhuk,42.0,0,0,1,0,1,0,0,0,1,...,0,12,4,2011,6,5,10,4,2009,764
87mebub9p4,41.0,1,0,0,0,0,1,0,0,1,...,1,9,3,2010,6,1,12,4,2009,279


# Adding New Data

## Clean and Transform session data

In [24]:
sessions = pd.read_csv('data/sessions.csv', header=0, index_col=False)

## extract the primary and secondary devices for each user

In [25]:
# Determine primary device
print("Determing primary device...")
sessions_device = sessions.loc[:, ['user_id', 'device_type', 'secs_elapsed']]
aggregated_lvl1 = sessions_device.groupby(['user_id', 'device_type'], as_index=False, sort=False).aggregate(np.sum)
idx = aggregated_lvl1.groupby(['user_id'], sort=False)['secs_elapsed'].transform(max) == aggregated_lvl1['secs_elapsed']
df_primary = pd.DataFrame(aggregated_lvl1.loc[idx , ['user_id', 'device_type', 'secs_elapsed']])
df_primary.rename(columns = {'device_type':'primary_device', 'secs_elapsed':'primary_secs'}, inplace=True)
df_primary = convert_to_binary(df=df_primary, column_to_convert='primary_device')
df_primary.drop('primary_device', axis=1, inplace=True)

Determing primary device...


In [26]:
# Determine Secondary device
print("Determing secondary device...")
remaining = aggregated_lvl1.drop(aggregated_lvl1.index[idx])
idx = remaining.groupby(['user_id'], sort=False)['secs_elapsed'].transform(max) == remaining['secs_elapsed']
df_secondary = pd.DataFrame(remaining.loc[idx , ['user_id', 'device_type', 'secs_elapsed']])
df_secondary.rename(columns = {'device_type':'secondary_device', 'secs_elapsed':'secondary_secs'}, inplace=True)
df_secondary = convert_to_binary(df=df_secondary, column_to_convert='secondary_device')
df_secondary.drop('secondary_device', axis=1, inplace=True)

Determing secondary device...


## Looping through the actions columns

In [27]:
# Count occurrences of value in a column
def convert_to_counts(df, id_col, column_to_convert):
    id_list = df[id_col].drop_duplicates()

    df_counts = df.loc[:,[id_col, column_to_convert]]
    df_counts['count'] = 1
    df_counts = df_counts.groupby(by=[id_col, column_to_convert], as_index=False, sort=False).sum()

    new_df = df_counts.pivot(index=id_col, columns=column_to_convert, values='count')
    new_df = new_df.fillna(0)

    # Rename Columns
    categories = list(df[column_to_convert].drop_duplicates())
    for category in categories:
        cat_name = str(category).replace(" ", "_").replace("(", "").replace(")", "").replace("/", "_").replace("-", "").lower()
        col_name = column_to_convert + '_' + cat_name
        new_df.rename(columns = {category:col_name}, inplace=True)
    return new_df

In [28]:
# Aggregate and combine actions taken columns
print("Aggregating actions taken...")
session_actions = sessions.loc[:,['user_id', 'action', 'action_type', 'action_detail']]
columns_to_convert = ['action', 'action_type', 'action_detail']
session_actions = session_actions.fillna('not provided')
first = True

for column in columns_to_convert:
    print("Converting " + column + " column...")
    current_data = convert_to_counts(df=session_actions, id_col='user_id', column_to_convert=column)

# If first loop, current data becomes existing data, otherwise merge existing and current
if first:
    first = False
    actions_data = current_data
else:
    actions_data = pd.concat([actions_data, current_data], axis=1, join='inner')

Aggregating actions taken...
Converting action column...
Converting action_type column...
Converting action_detail column...


## Combine Data Sets

In [29]:
# Merge device datasets
print("Combining results...")
df_primary.set_index('user_id', inplace=True)
df_secondary.set_index('user_id', inplace=True)
device_data = pd.concat([df_primary, df_secondary], axis=1, join="outer")

# Merge device and actions datasets
combined_results = pd.concat([device_data, actions_data], axis=1, join='outer')
df_sessions = combined_results.fillna(0)

# Merge user and session datasets
#all_df.set_index('id', inplace=True)
all_df = pd.concat([all_df, df_sessions], axis=1, join='inner')

Combining results...


# Create Model

## choosing an Algorithm
- Decision tree
- k-nearest neighbors
- neural networks
- svm


## Cross Validation

### Training model

In [32]:
# Prepare training data for modelling
#train_df.set_index('id', inplace=True)
train_df = pd.concat([train_df['country_destination'], all_df], axis=1, join='inner')

id_train = train_df.index.values
labels = train_df['country_destination']
le = preprocessing.LabelEncoder()
y = le.fit_transform(labels)
X = train_df.drop('country_destination', axis=1, inplace=False)

In [38]:
# Grid Search - Used to find best combination of parameters
XGB_model = xgb.XGBClassifier(objective='multi:softprob', subsample=0.5, colsample_bytree=0.5, seed=0)
param_grid = {'max_depth': [3, 4, 5], 'learning_rate': [0.1, 0.3], 'n_estimators': [25, 50]}
model = grid_search.GridSearchCV(estimator=XGB_model, param_grid=param_grid, scoring='accuracy', verbose=10, n_jobs=1, iid=True, refit=True, cv=3)

model.fit(X, y)
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] learning_rate=0.1, max_depth=3, n_estimators=25 .................
[CV]  learning_rate=0.1, max_depth=3, n_estimators=25, score=0.6987525903051481, total=  11.0s
[CV] learning_rate=0.1, max_depth=3, n_estimators=25 .................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.2s remaining:    0.0s


[CV]  learning_rate=0.1, max_depth=3, n_estimators=25, score=0.6909977646819752, total=  11.2s
[CV] learning_rate=0.1, max_depth=3, n_estimators=25 .................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   24.6s remaining:    0.0s


[CV]  learning_rate=0.1, max_depth=3, n_estimators=25, score=0.6843774137160047, total=  11.8s
[CV] learning_rate=0.1, max_depth=3, n_estimators=50 .................


...

[CV]  learning_rate=0.1, max_depth=4, n_estimators=25, score=0.6908410911012642, total=  13.5s
[CV] learning_rate=0.1, max_depth=4, n_estimators=50 .................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  2.6min remaining:    0.0s


[CV]  learning_rate=0.1, max_depth=4, n_estimators=50, score=0.7033846653935232, total=  24.1s
[CV] learning_rate=0.1, max_depth=4, n_estimators=50 .................
[CV]  learning_rate=0.1, max_depth=4, n_estimators=50, score=0.6973379394432025, total=  24.0s
[CV] learning_rate=0.1, max_depth=4, n_estimators=50 .................
[CV]  learning_rate=0.1, max_depth=4, n_estimators=50, score=0.6979958534899793, total=  24.3s
[CV] learning_rate=0.1, max_depth=5, n_estimators=25 .................
[CV]  learning_rate=0.1, max_depth=5, n_estimators=25, score=0.6984275324042095, total=  18.6s
[CV] learning_rate=0.1, max_depth=5, n_estimators=25 .................
[CV]  learning_rate=0.1, max_depth=5, n_estimators=25, score=0.6957528957528958, total=  17.0s
[CV] learning_rate=0.1, max_depth=5, n_estimators=25 .................
[CV]  learning_rate=0.1, max_depth=5, n_estimators=25, score=0.6919386966949876, total=  18.8s
[CV] learning_rate=0.1, max_depth=5, n_estimators=50 .................
[CV]

[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed: 12.3min finished


Best score: 0.701
Best parameters set:
	learning_rate: 0.1
	max_depth: 5
	n_estimators: 50


### Make predictions

In [39]:
# Prepare test data for prediction
#test_df.set_index('id', inplace=True)
test_df = pd.merge(test_df.loc[:,['date_first_booking']], all_df, how='left', left_index=True, right_index=True, sort=False)
X_test = test_df.drop('date_first_booking', axis=1, inplace=False)
X_test = X_test.fillna(-1)
id_test = test_df.index.values

# Make predictions
y_pred = model.predict_proba(X_test)

In [40]:
#Taking the 5 classes with highest probabilities
ids = []  #list of ids
cts = []  #list of countries
for i in range(len(id_test)):
    idx = id_test[i]
    ids += [idx] * 5
    cts += le.inverse_transform(np.argsort(y_pred[i])[::-1])[:5].tolist()

#Generate submission
print("Outputting final results...")
sub = pd.DataFrame(np.column_stack((ids, cts)), columns=['id', 'country'])
sub.to_csv('data/mysubmission.csv', index=False)

Outputting final results...
